## Objetivo: 

Temos como principal objetivo da competição, fazer a predição de vendas das lojas e departamentos do Walmart, utilizando variáveis como tamanho da loja, se existe feriado ou não naquela semana entre outras que estão disponíveis nas bases fornecidas pela própria empresa e listadas abaixo


*Abaixo, segue descrição dos dados.*


stores.csv: This file contains anonymized information about the 45 stores, indicating the type and size of store.

train.csv: This is the historical training data, which covers to 2010-02-05 to 2012-11-01. Within this file you will find the following fields:

    Store - the store number
    Dept - the department number
    Date - the week
    Weekly_Sales -  sales for the given department in the given store
    IsHoliday - whether the week is a special holiday week

test.csv: This file is identical to train.csv, except we have withheld the weekly sales. You must predict the sales for each triplet of store, department, and date in this file.

features.csv: This file contains additional data related to the store, department, and regional activity for the given dates. It contains the following fields:

    Store - the store number
    Date - the week
    Temperature - average temperature in the region
    Fuel_Price - cost of fuel in the region
    MarkDown1-5 - anonymized data related to promotional markdowns that Walmart is running. MarkDown data is only available after Nov 2011, and is not available for all stores all the time. Any missing value is marked with an NA.
    CPI - the consumer price index
    Unemployment - the unemployment rate
    IsHoliday - whether the week is a special holiday week

For convenience, the four holidays fall within the following weeks in the dataset (not all holidays are in the data):

- Super Bowl: 12-Feb-10, 11-Feb-11, 10-Feb-12, 8-Feb-13
- Labor Day: 10-Sep-10, 9-Sep-11, 7-Sep-12, 6-Sep-13
- Thanksgiving: 26-Nov-10, 25-Nov-11, 23-Nov-12, 29-Nov-13
- Christmas: 31-Dec-10, 30-Dec-11, 28-Dec-12, 27-Dec-13 

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import datetime as dt
from matplotlib import pylab
plt.rcParams["figure.figsize"] = [16,9]
from sklearn.preprocessing import StandardScaler
import seaborn as sns 
from sklearn.preprocessing import OneHotEncoder
from sklearn import base
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer
from h2o.automl import H2OAutoML

## Importando biblioteca h2o para utilizar o Target Encoder:
import h2o
h2o.init()
from h2o.estimators import H2OTargetEncoderEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Importando dados 

In [ ]:
train_df = pd.read_csv("../input/walmart-recruiting-store-sales-forecasting/train.csv.zip")
features_df = pd.read_csv("../input/walmart-recruiting-store-sales-forecasting/features.csv.zip")
stores_df = pd.read_csv("../input/walmart-recruiting-store-sales-forecasting/stores.csv")
test_df = pd.read_csv("../input/walmart-recruiting-store-sales-forecasting/test.csv.zip")

## Convertendo nome das colunas para letra minuscula

In [ ]:
def columns_to_lower(data):
    return data.columns.str.lower()


train_df.columns = columns_to_lower(train_df)
features_df.columns = columns_to_lower(features_df)
stores_df.columns = columns_to_lower(stores_df)
test_df.columns = columns_to_lower(test_df)

## Join entre os dataframes disponíveis

In [ ]:
dataset = pd.DataFrame()

print(train_df.shape)
dataset = train_df.merge(features_df.drop("isholiday",axis = 1) , how = "left" , on = ["date","store"])
print(dataset.shape)
dataset = dataset.merge(stores_df,how='left', on = ["store"])
print(dataset.shape)

dataset['store'] = dataset['store'].astype('object')
dataset['dept']  = dataset['dept'].astype('object')

## Convertendo: 

* Coluna date para formato de data 

* Colunas store and dept para categorical

In [ ]:
dataset['year'] = dataset['date'].str[:4]
dataset['date']  = pd.to_datetime(dataset['date'])
dataset['month'] = dataset['date'].dt.month

features_df['date'] = pd.to_datetime(features_df['date'])
features_df['week'] = features_df['date'].dt.isocalendar()['week']
features_df['year'] = features_df['date'].dt.isocalendar()['year']

## Verificando valores faltantes 

In [ ]:
pd.DataFrame({'missing_quantity' : dataset.isnull().sum().sort_values(ascending = False),
              'missing_pct' : dataset.isnull().mean().sort_values(ascending = False)})

Todas as colunas Markdown, apresentam pelo menos 60% de valores faltantes, o que pode o que pode ser um problema (ou não). Nesse caso, onde as colunas indicam a presença ou não de uma campanha de desconto, podemos criar uma coluna indicando se houve desconto ou não naquela seamana

## Analisando a variável resposta 

In [ ]:
dataset['weekly_sales'].describe().T.to_frame()

Analisando as medidas resumo da variável resposta, é possível notar a presençã de valores negativos, o que é estranho por se tratar de vendas. Para não "alterar" os dados vou seguir com os valores nesse formato mesmo. Porém o mais indicado, nesse caso, é contactar o responsável pelo fornecimento dos dados para ter um melhor entendimento do porque desses valores.

*Ps: Uma outra abordagem possível, seria aplicar o modulo e utilizar os valores absolutos, porém estariamos a alterar os dados sem um conhecimento para justificar tal ação*

In [ ]:
def target_description_plot(df):

    plt.figure(figsize = (25,8))

    plt.subplot(131)
    sns.lineplot(x = 'date' , y = 'weekly_sales' , data = df )
    plt.title("Time Series from Weekly Sales")

    plt.subplot(132)
    sns.distplot(df['weekly_sales'],hist = True )
    plt.title("Density from Weekly Sales")

    plt.subplot(133)
    sns.boxplot(y = df['weekly_sales'])
    plt.title("Boxplot from Weekly Sales") ; 

## Análise univariada da variável resposta 

In [ ]:
target_description_plot(dataset)

Com base nos gráficos acima, podemos notar: 

* As semanas que antecedem o natal e o feriado de thanksaving, já apresentam um aumento nas vendas, podemos pensar em alguma variável que indique um número n de semanas anteriores a semana desses feriados (ou até mesmo dos demais feriados)

* A distribuição apresenta assimetria positiva

* Com base no boxplot, e considerando que um possível valor discrepante pode existir quando o valor da observação é maior que 3Quartil + 1.5 * IQD (1Quartil - 1.5 * IQD). O que pode impactar, inicialmente, no valor da média de vendas (vamos gerar o gráfico do valor médio e mediano de vendas para verficiar isso).

In [ ]:
dataset.groupby(['date'])['weekly_sales'].mean().plot()
dataset.groupby(['date'])['weekly_sales'].median().plot()
plt.legend(['Media','Mediana']) 
plt.title("Media e Mediana de vendas ao longo do tempo")
plt.ylabel("Vendas"); 


Como esperado, os possíveis outliers acabam por elevar a média. Vale a pena estar considerando a mediana nas nossas análises para variável resposta,por ser uma medida mais robusta em comparação a média

## Analise univariada:

In [ ]:
for i in ['temperature','fuel_price', 'cpi', 'unemployment']:

    plt.figure(figsize = (30,6))

    plt.subplot(131)
    sns.lineplot(x = 'date', y =  i , data = features_df,ci=None)
    plt.title( i + " ao longo do tempo"  )
    plt.xticks(rotation = 90)
    
    plt.subplot(132)
    sns.boxplot(y =  i , data = features_df)
    plt.title("Boxplot " + i)
   

    plt.subplot(133)
    sns.distplot(features_df[i].dropna())

    plt.show() ; 

Alguns pontos: 

- A variável temperatura apresenta um comportamento sazonal ao longo dos anos (como esperado). Uma vez que o inverno americano é bem rigoroso, chegando a dificultar a locomoçã, as vendas podem ser impactadas por esse motivo, além disso vale a pena criar variáveis dummy indicando a estação do ano ?

- A variável fuel_price, apresenta um compartamento sazonal também, apresentando maiores valores próximo ao mês de maio. Além disso, ela sofreu uma um aumento relevante do ano de 2010 para 2011/2012. O que pode impactar as vendas, uma vez que o custo de locomoção/logística aumenta.

- CPI mede a variação de preços a partir da perspectiva do consumidor.
  É uma maneira fundamental para medir as variações de tendências de compra e a inflação nos Estados Unidos. Valores superiores aos esperados devem ser considerados como positivo/alta para o USD   (sendo o caminho comum para lutar contra a inflação o aumento das taxas, o que pode atrair o investimento estrangeiro), enquanto valores inferiores aos esperados devem ser considerados como    negativos/baixos para o USD. (fonte:[https://br.investing.com/economic-calendar/cpi-733] ). 
  Uma vez que os valores do CPI estão aumentando, seria esse um bom sinal para as vendas  e teria ele influência com as vendas semanais do Walmart ?
  
- A variável Unemployement (taxa de desemprego) apresenta queda ao longo dos anos, ou seja, maior parte da população está empregada e pode estar apta a realizar compras. Vale verificar se existe alguma correlação com as vendas! 

## Variáveis categóricas: 

In [ ]:
stores_df['type'].value_counts().plot(kind='bar') 
plt.title("Quantidade de cada tipo de loja")  
plt.ylabel("Quantidade") ; 

In [ ]:
plt.figure(figsize = (30,6))

plt.subplot(121)
sns.boxplot(y = stores_df['size'])
plt.title("Boxplot - Tamanho da Loja")

plt.subplot(122)
sns.distplot(stores_df['size'])
plt.title("Densidade - Tamanho da loja"); 

Nenuma loja aparenta ter um valor discrepante em relação as demais (sem presençã de possíveis outliers)

## Análise Bivariada:

## Matriz de correlação

In [ ]:
num_columns = dataset.select_dtypes(['int','float64']).columns
# multicolinearidade
plt.figure(figsize = (20,5))
sns.heatmap(data = dataset[num_columns].corr() , annot=True) ;

* A variável weekly sales não apresenta nenhum indício de correlação linear forte com as demais variávei numéricas, a variável size é a que apresenta o maior indice (0.24). Mas além de correlação linear, será que alguma outra correlação pode existir nos dados ?

* As variáveis Markdown1 e Markdown4 apresentam uma correlação forte (-0.82), possivelmente vamos utilizar apenas uma delas no processo de modelagem



## SccaterPlot das variáveis continuas

In [ ]:
def plot_scatter( var ):
    if var != 'weekly_sales':
        sns.scatterplot(x = var , y = 'weekly_sales' , data = dataset)
        plt.title(var + " weekly_sales")
        plt.show() ; 

In [ ]:
for i in num_columns:
    plot_scatter(i) ; 

Com base nos gráficos acima não foi possível identificar nenhuma relação explicita entre as variáveis. O unico ponto que pode ser relevante aqui, é em relação a temperatura, onde podemos observar um certo aumento nas vendas a partir dos 20 Graus mas nada muito relevante que possa gerar algum insight.

## Variáveis categóricas

In [ ]:
def cat_plot(var_cat):
    sns.boxplot(x = var_cat , y = 'weekly_sales' , data = dataset) 
    plt.title("Boxplot weekly sales por " + var_cat) 
    plt.show(); 

In [ ]:
categorical_features = dataset.select_dtypes(['object']).columns
for i in categorical_features:
    cat_plot(i)

* A maioria das lojas apresenta uma mediana de vendas proximas, algumas com maior presença de possíveis outlier's.

* Já para os departamentos, é possível notar que alguns desses apresentam uma mediana maior em relação a outros. Além disso, a elevada quantidade de departamentos pode levar ao overfitting, podemos pensar em agrupa-los de alguma maneira. 

* Loja do tipo A, apresentam uma média e uma mediana de vendas maior em comparação as demais. Vamos verificar o tamanho da loja, por tipo de loja 

## Tamnho da loja por tipo de loja e vendas por tipo de loja 


In [ ]:
plt.figure(figsize = (25,8))
plt.subplot(121)
sns.lineplot(x = 'date',y = 'weekly_sales',hue='type',data=dataset.groupby(['date','store','type'])['weekly_sales'].median().reset_index(),ci = None)


plt.subplot(122)
sns.boxplot(x = 'type',
            y = 'size' ,
            data = dataset) 
plt.title("Boxplot: Tamanho da loja por tipo de loja "); 

As lojas do tipo A em gerale apresentam a venda mediana maior ao longo do tempo, são maiores comparadas com as demais, já a lojas do tipo C são as menores e apresentam a menor mediana de vendas ao longo do tempo. Seria algum tipo de loja Premium ou Express (caso exista esse tipo de loja na rede) 

## Criando estação do ano:

In [ ]:
def getSeason(x):
    
    if (x > 11 or x <= 3):
        return "winter"
    elif ( x == 4 or x == 5):
        return "spring"
    elif (x >=6 and x <= 9):
        return "summer"
    else:
        return "fall"
    
dataset['season'] = dataset['month'].apply(getSeason)

## Variável: Departamento

In [ ]:
"Como visto, a variável departamento apresenta {} categorias distintas para tentar melhorar isso, vamos tentar agrupa-las de alguma maneira".format(dataset['dept'].nunique())

## Modelagem 

Usualmente utilizamos a função train_test_split para verificar a perfomance do modelo na base de teste, nesse caso vamos separar com base em datas

A distribuição não ficou como é feito geralmente (70% treino/30%teste), pois é interessante avaliar como o modelo irá performar em feriados, principalmente no natal e thanksaving

In [ ]:
train = dataset.query("date < '2011-10-01'")
test  = dataset.query("date >= '2011-10-01'") 
train.fillna(0,inplace=True)
test.fillna(0,inplace=True)

## Criando variável grupo departamento:

In [ ]:
train.groupby(['dept'])['weekly_sales'].median().sort_values().plot(kind='bar') ; 

In [ ]:
def group_dept(x):
    if x < 10000:
        return 'C'
    elif x <20000:
        return 'B'
    else:
        return 'A'

    
    
train_dept = train.groupby(['dept'])['weekly_sales'].median().sort_values().reset_index()
train_dept['grupo_dept'] = train_dept['weekly_sales'].apply(group_dept)
train = train.merge(train_dept[['dept','grupo_dept']].drop_duplicates(),on = 'dept')
test  = test.merge(train_dept[['dept','grupo_dept']].drop_duplicates(),on='dept')

Para agrupar a variável departamento, vamos nos basear na mediana de vendas de cada departamento agrupando-os com base e valores próximos. Ainda poderia ser pensado em clusterizarmos esses departmentos utilizando algum método de agrupamento (K-Means,DbScan, entre outros...). Mas por hora vamos seguir dessa maneira

## Função para calculo do erro personalizado

In [ ]:
def WMAE(data, real, predicted):
    weights = data['isholiday_True'].apply(lambda x: 5 if x == 1 else 1)

    return np.round(np.sum(weights*abs(real-predicted))/(np.sum(weights)), 2)



# Erro personalizado para utilizar no grid search:
erro = make_scorer(WMAE,greater_is_better = False)

## Preparando o dataset para testar alguns modelos 

In [ ]:
def prepdata(data ,categorical_var,continuous_var):
    
    
    onehot = OneHotEncoder(handle_unknown='error',sparse = False,drop='first')
    onehot.fit(data[categorical_var])
    df_cat = pd.DataFrame( onehot.transform( data[categorical_var] ) , columns = onehot.get_feature_names(categorical_var) )
    
    Standard = StandardScaler()
    Standard.fit(train[numerical_features]) 
    df_standard = pd.DataFrame( Standard.transform(data[numerical_features]) , columns = list(numerical_features) )
    
    data_train = pd.concat( [df_standard,df_cat] , axis = 1 )
    data_train['target'] = data[target]
    
    X_train = data_train.drop('target',axis=1)
    y_train = data_train['target']
    
    
    return X_train,y_train

def fit_model(X,y):
    
    fit_LR = LinearRegression().fit( X , np.ravel(y) )
    fit_knn = KNeighborsRegressor( n_neighbors = 10 ).fit( X, np.ravel(y) )
    fit_Rf = RandomForestRegressor(min_samples_split=50,max_depth=10).fit( X , np.ravel(y) )
    fit_GB = GradientBoostingRegressor(min_samples_split=50,max_depth=10).fit( X , np.ravel(y) )
    
    
    return fit_LR,fit_knn,fit_Rf,fit_GB


def test_model(data ,categorical_var,continuous_var):
    
    onehot = OneHotEncoder(handle_unknown='error',sparse = False,drop='first')
    onehot.fit(train[categorical_var])
    df_cat = pd.DataFrame( onehot.transform( data[categorical_var] ) , columns = onehot.get_feature_names(categorical_var) )
    
    Standard = StandardScaler()
    Standard.fit(data[numerical_features]) 
    df_standard = pd.DataFrame( Standard.transform(data[numerical_features]) , columns = list(numerical_features) )
    
    data_test = pd.concat( [df_standard,df_cat] , axis = 1 )
    data_test['target'] = data[target]
    
       
    return data_test

## Scorando primeiros modelos:

In [ ]:

categorical_features = ['store','isholiday','grupo_dept','type']
numerical_features = ['temperature','fuel_price','markdown1','markdown2','markdown3','markdown5','cpi','unemployment','size']

target = 'weekly_sales'
X_train,y_train = prepdata(train,categorical_features,numerical_features)


fitted_models = []
fitted_models = fit_model(X_train,y_train)

##################################################################################################################################

test_df = test_model(test,categorical_features,numerical_features)
X_test = test_df.drop("target",axis=1)
y_test = test_df.drop("target",axis=1)

##################################################################################################################################

list_erro = []
list_model = []
for i in fitted_models:
        list_erro.append( WMAE(test_df,test_df['target'],i.predict(X_test) ))
        list_model.append( str(i) )


        
display( pd.DataFrame(list_erro,list_model) )

Utilizando todas as variáveis fornecidas, podemos observar que o modelo random forest apresenta melhor desempenho comparado aos demais. Vamos veriricar a importância das variáveis do Random Forest e em seguida ajustar os mesmos modelos com a adição de algumas features como: 

* Tempo até o feriado mais recente ( a ser criada )

* Estação do ano

In [ ]:
pd.DataFrame(fitted_models[2].feature_importances_,X_train.columns).sort_values(by = 0 , ascending = False).plot(kind = 'bar') ; 

A variável grupo_dept (referente ao agrupamaneto dos departamentos), ter maior importancia no nosso modelo. Já a variável Markdown apresenta menor importancia, podemos incluisive tentar o ajuste de um modelo sem a mesma!

## Scorando modelo reduzido adicionado da feature estação do ano

In [ ]:

categorical_features = ['store','isholiday','season','grupo_dept','type']
numerical_features = ['temperature','fuel_price','cpi','unemployment','size']
target = 'weekly_sales'

##################################################################################################################################

X_train,y_train = prepdata(train,categorical_features,numerical_features)
fitted_models = []
fitted_models = fit_model(X_train,y_train)

##################################################################################################################################

test_df = test_model(test,categorical_features,numerical_features)
X_test = test_df.drop("target",axis=1)
y_test = test_df.drop("target",axis=1)

list_erro = []
list_model = []
for i in fitted_models:
        list_erro.append( WMAE(test_df,test_df['target'],i.predict(X_test) ))
        list_model.append( str(i) )


        
display(pd.DataFrame(list_erro,list_model) )


Reduzindo o número de variáveis e utilizando a variável estação do ano, temos uma redução no erro, principalmente no modelo de regressão linear, indicando que a estação do ano (de forma mais geral, existe uma sazonalidade) que existe nos dados. Indicando que um trabalho de feature engineering pode agregar muito ao nosso modelo!

## Abaixo também temos um exemplo de uso do framework H2O

In [ ]:
x_columns = ['temperature',
             'fuel_price',
             'markdown1',
             'markdown2',
             'markdown3',
             'markdown5',
             'cpi',
             'unemployment',
             'size',
             'store',
             'isholiday',
             'season',
             'grupo_dept',
             'type']

y_column = 'weekly_sales'

In [ ]:
# # Identify predictors and response
# #train = h2o.H2OFrame(train)
# x = x_columns
# y = y_column


# # Run AutoML for 20 base models (limited to 1 hour max runtime by default)
# aml = H2OAutoML(max_models= 8 , seed=1)
# aml.train(x=x, y=y, btraining_frame=train)

# # View the AutoML Leaderboard
# lb = aml.leaderboard
# lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

## Possiveis próximos passos

* Utilizar alguma método de otimização de parâmetros, GridSearchCV, RandomizerSearchCv entre outros. 

* Utilizar algoritmos mais complexos como XgBoosting,CatBoosting,Adaptative Boosting entre outros. Que dos testados acima, do ponto de vista de "implementação", é apenas organizar os dados da maneira aceita e seguir o mesmo processo de modelagem.

* Criação de novas features que podem melhorar a qualidade do nosso modelo.
